In [2]:
!pip install -U -q transformers datasets bitsandbytes trl peft

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#model_name = "cognitya/llama-2-1B-instruct-edited"
model_name = "meta-llama/Llama-3.2-1B-Instruct"

config_8bit = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=config_8bit,
    device_map="auto",
    trust_remote_code=True,
)

In [5]:
model_8bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMS

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", trust_remote_code=True)

In [8]:
from datasets import load_dataset
dataset = load_dataset("MattCoddity/dockerNLcommands")

README.md:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

06102023.json:   0%|          | 0.00/543k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2415 [00:00<?, ? examples/s]

In [10]:
dataset['train'][0]

{'input': 'Give me a list of containers that have the Ubuntu image as their ancestor.',
 'output': "docker ps --filter 'ancestor=ubuntu'",
 'instruction': 'translate this sentence in docker command'}

In [11]:
from datasets import DatasetDict
train_test_split = dataset['train'].train_test_split(test_size=0.2,seed=42)
dataset = DatasetDict(
    {
        'train':train_test_split['train'], 
        'validation' :train_test_split['test']
    }
)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 483
    })
})

In [13]:
def chat_template(example):
    mesagges =[
        {'role':'system', 'content':example['instruction']},
        {'role':'user', 'content':example['input']},
        {'role':'assistant', 'content':example['output']},
    ]

    return {'text':mesagges}

dataset = dataset.map(chat_template)
dataset

Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 483
    })
})

In [14]:
dataset['train']['text'][0]

[{'content': 'translate this sentence in docker command', 'role': 'system'},
 {'content': 'Find the repository, tag, and ID of the images that were created before the latest nginx image.',
  'role': 'user'},
 {'content': 'docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"',
  'role': 'assistant'}]

In [17]:
tokenizer_mistral = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer_mistral.apply_chat_template(dataset['train']['text'], tokenize=False)

['<s> [INST] translate this sentence in docker command\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image. [/INST] docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"</s>',
 "<s> [INST] translate this sentence in docker command\n\nPlease show me the Docker containers that have exited and are related to the mongo image. [/INST] docker ps -a --filter 'status=exited' --filter 'ancestor=mongo'</s>",
 '<s> [INST] translate this sentence in docker command\n\nTo gain access, submit your username and password from the text file for successful login. [/INST] "docker login --username=johndoe --password-stdin < ~/mypassword.txt"</s>',
 '<s> [INST] translate this sentence in docker command\n\nCease the execution of confusioner. [/INST] "docker kill confusioner"</s>',
 "<s> [INST] translate this sentence in docker command\n\nList all running containers created later than 3e33ad9a0b3e. [/INST] docker ps --filter 'since=3e3

In [18]:
tokenizer.apply_chat_template(dataset['train']['text'][0],tokenize=False)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 05 Apr 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"<|eot_id|>'

In [19]:
def apply_chat_temp(example):
    new_text = tokenizer.apply_chat_template(example['text'],tokenize=False)

    return {'text':new_text}


dataset = dataset.map(apply_chat_temp)
dataset

Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 483
    })
})

In [20]:
dataset['train']['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 05 Apr 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"<|eot_id|>'

In [21]:
tokenizer(dataset['train']['text'][0])

{'input_ids': [128000, 128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 2304, 5186, 220, 2366, 20, 271, 14372, 420, 11914, 304, 27686, 3290, 128009, 128006, 882, 128007, 271, 10086, 279, 12827, 11, 4877, 11, 323, 3110, 315, 279, 5448, 430, 1051, 3549, 1603, 279, 5652, 71582, 2217, 13, 128009, 128006, 78191, 128007, 271, 29748, 5448, 482, 69, 330, 15145, 28, 74661, 25, 19911, 1, 1198, 2293, 48319, 13, 4727, 39254, 3052, 13, 5786, 39254, 3052, 13, 926, 24275, 128009], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
def tokenize_fn(example):
    return tokenizer(example['text'])


tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=['input','output','instruction','text'])

tokenized_dataset

Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 483
    })
})

In [23]:
#The data collector ensures that all input vectors are aligned to the same size, as the GPU expects inputs of uniform dimensions. To achieve this, the data collector pads each vector to match the size of the largest one.

In [24]:
from transformers import DataCollatorForLanguageModeling

data_collector = DataCollatorForLanguageModeling(tokenizer,mlm=False, return_tensors="pt")

In [26]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|eot_id|>'